In [35]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit

In [2]:
import sys

In [3]:
sys.path.append("../src")

In [4]:
from utils import load_obj, dict_to_df, make_df, make_lama_df, Metrics, form_Xy

In [5]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/home/gavrilenkoa/anaconda3/envs/bind_predict/lib/python3.11/site-packages/lightautoml/text/tokenizer.py:21: UserWarning: 'nltk' - package isn't installed
  warnings.warn("'nltk' - package isn't installed")
/home/gavrilenkoa/anaconda3/envs/bind_predict/lib/python3.11/site-packages/lightautoml/transformers/text.py:22: UserWarning: 'gensim' - package isn't installed
  warnings.warn("'gensim' - package isn't installed")
/home/gavrilenkoa/anaconda3/envs/bind_predict/lib/python3.11/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


In [24]:
df = pd.read_csv("../data/ready_data/df_3.csv")

In [26]:
SEED = 42

In [27]:
embeddings = load_obj("../data/embeddings/df_3.pkl")

In [28]:
embeddings_df = dict_to_df(embeddings)

In [29]:
df = df.merge(embeddings_df, on="identifier")

In [30]:
fraction = len(df.clusters.unique()) / len(df)
print(f"Доля уникальных кластеров относительно сиквенсов {fraction}")

Доля кластеров относительно сиквенсов 0.3291292629760918


In [31]:
i = 0
grouped = df.groupby("clusters")
for _, group in grouped:
    if len(group.label.unique()) > 1:
        i += 1
fraction = np.round((i / len(grouped)) * 100, 2)
print(f"Доля кластеров в которых есть сиквенсы с разными лейблами {fraction} %")

Доля кластеров в которых есть сиквенсы с разными лейблами 0.09 %


In [32]:
X, y, clusters = form_Xy(df, clusters="Yes")

In [36]:
gss = GroupShuffleSplit(n_splits=1, train_size=.65, random_state=SEED)

In [37]:
for train_idx, test_idx in gss.split(X, y, groups=clusters):
    X_train = X[train_idx]
    X_test = X[test_idx]

    y_train = y[train_idx]
    y_test = y[test_idx]

    clusters_train = clusters[train_idx]
    clusters_test = clusters[test_idx]

In [38]:
df_train = make_lama_df(X_train, y_train, clusters=clusters_train)

In [40]:
roles = {
    "target": "label",
    "group": "clusters"
}

In [41]:
task = Task("binary")

In [42]:
automl = TabularAutoML(
    task = task,
    reader_params = {'random_state': SEED})

In [43]:
oof_pred = automl.fit_predict(
    df_train,
    roles = roles)

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric perio

In [44]:
valid_prob = oof_pred.data[:, 0]
valid_pred = (valid_prob > 0.5) * 1

In [45]:
metrics = Metrics(y_train, valid_pred, valid_prob)
metrics.get_metrics()

,accuracy,sensitivity,specificity,precision,AUC,MCC
0,0.997122,0.994461,0.999725,0.999717,0.999943,0.994257


In [46]:
df_test = make_lama_df(X_test, y_test, clusters=clusters_test)

In [47]:
test_pred = automl.predict(df_test)
test_prob = test_pred.data.reshape(-1, )
test_pred = (test_pred.data[:, 0] > 0.5) * 1

In [48]:
metrics = Metrics(y_test, test_pred, test_prob)
metrics.get_metrics()

,accuracy,sensitivity,specificity,precision,AUC,MCC
0,0.997127,0.994396,0.999802,0.999797,0.999892,0.994267


#### Test on the benchmark test data

In [49]:
df = make_df("../data/embeddings/overall_data.pkl", "../data/DBP_papers_data/overall_data.csv")
pdb20000 = df[df["identifier"].str.contains("pdb20000")]
X_test, y_test = form_Xy(pdb20000)
df_test = make_lama_df(X_test, y_test)
test_pred = automl.predict(df_test)
test_prob = test_pred.data.reshape(-1, )
test_pred = (test_pred.data[:, 0] > 0.5) * 1
metrics = Metrics(y_test, test_pred, test_prob)
metrics.get_metrics()

,accuracy,sensitivity,specificity,precision,AUC,MCC
0,0.513,0.0261,0.9999,0.996183,0.841428,0.114333


In [50]:
pdb186 = df[df["identifier"].str.contains("pdb186")]
X_test, y_test = form_Xy(pdb186)
df_test = make_lama_df(X_test, y_test)
test_pred = automl.predict(df_test)
test_prob = test_pred.data.reshape(-1, )
test_pred = (test_pred.data[:, 0] > 0.5) * 1
metrics = Metrics(y_test, test_pred, test_prob)
metrics.get_metrics()

/home/gavrilenkoa/anaconda3/envs/bind_predict/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,accuracy,sensitivity,specificity,precision,AUC,MCC
0,0.5,0.0,1.0,0.0,0.659845,0.0
